In [102]:
import numpy as np
from random import random, randint, choice

In [103]:
def V(s, w):
    if s == 6:
        return (w[s] + 2*w[7])
    return (2*w[s] + w[7])

In [104]:
def nabla_V(s, w = []):
    if s < 0 or s > 6:
        return None
    grad = np.zeros(8)
    if s == 6:
        grad[s] = 1
        grad[7] = 2
    else:
        grad[s] = 2
        grad[7] = 1
        
    return grad

In [105]:
# The transition reward is zero, this
# makes converting the value_state function
# into the value_action function trivial.
def Q(s, s_next, w, gamma=0.99):
    # a=action -> is the next_state
    # R + V(s_next)
    return 0 + gamma*V(s_next,w)

def nabla_Q(s, s_next, w, gamma=0.99):
    return gamma*nabla_V(s_next, w)

In [113]:
print("Q(s, a)=" + str(Q(0,1,np.ones((8)))))
print("nabla_Q(s, a)= \n"+ str(nabla_Q(1,2,np.ones((8)))))

Q(s, a)=2.9699999999999998
nabla_Q(s, a)= 
[0.   0.   1.98 0.   0.   0.   0.   0.99]


In [116]:
def b_dist(s):
    # if in one of the 6 upper states:
    #    6/7 chance it goes to one of the 6 other states
    #    1/7 change it goes to the lower state 7
    # if in the lower state
    dist = np.ones(7)
    return dist*(1.0/7.0)

# we assume that the solid lines are valid transitions
def b_eval(s):   
    return np.random.choice(range(7), p=b_dist(s))

In [120]:
print("b_dist(0)[0]=" + str(b_dist(0)[0]))
print("b_eval(s)=" + str(b_eval(0)))

b_dist(0)[0]=0.14285714285714285
b_eval(s)=4


In [174]:
def pi_dist(s):
    dist = np.zeros(7)
    dist[6] = 1
    return dist

def pi_eval(s):
    return np.random.choice(range(7), p=pi_dist(s))

In [184]:
def pi_dist_epsilon(s):
    epsilon = 1e7
    dist = np.ones(7)
    dist = dist*epsilon
    dist[6] = 1 - 6*epsilon
    return dist

def pi_epsilon_eval(s):
    return np.random.choice(range(7), p=pi_dist_epsilon(s))

In [186]:
w = np.random.rand(8)
s = 1
alpha = 1
gamma = 0.99

R = 0 # no transition cost
for i in range(10):
    a = b_eval(s)
    
    p_a_target_policy = pi_dist_epsilon(s)[a]
    p_a_behavior_policy = b_dist(s)[a]
    rho = p_a_target_policy/p_a_behavior_policy
    # If b is not epsilon but greedy, then rho is zero unless
    # the behavior_policy pickes the same action. 
    # Which makes this equivalent to a wastefull on-policy algorithm.
    
    s_next = a # the action a represents the next state
    Q_max_costs = np.array([Q(s_next, a_, w) for a_ in range(7)])
    a_max = np.argmax(Q_max_costs)
    
    TD = R + gamma* Q_max_costs[a_max] - Q(s, a, w)
    w_old = w
    w = w + alpha*rho*TD*nabla_Q(s, a, w, gamma)
    
    print(np.linalg.norm(w_old-w))
    
print(w)

287009523.5446869
984543120127792.6
2.7086158590849803e+23
9.291500378438898e+29
6.374617806335476e+35
1.7275090013868097e+44
4.6815087797592254e+52
1.2686778777572658e+61
3.4380872219125723e+69
1.1793842510391878e+76
[ 5.48367447e-01  9.87079203e-01  1.13473999e+61 -1.05487304e+76
  5.41194515e-01  1.54513102e+44  2.32901398e-01 -5.27436518e+75]
